In [ ]:
#default_exp dag.buoy_data

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#export
from huey.imports import *
from huey.dag.airflow_imports import *

[2020-02-07 09:11:08,876] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=24528


In [ ]:
#export
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2019, 7, 24),
    'email': ['heyhueyapp@gmail.com'],
    'email_on_failure': True,
    'email_on_retry': True,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
}

In [ ]:
#export
# @hourly
dag = DAG(
    'huey_import_buoy_data', \
    default_args=default_args, \
    schedule_interval="0 * * * *", \
    catchup=False
)

In [ ]:
dag

<DAG: huey_import_buoy_data>

# Import realtime wave data

In [ ]:
#export
from huey.data.importer.buoy_data import import_buoy_realtime_wave_detail

In [ ]:
#export
def run_import_buoy_realtime_wave_detail():
    station_id = "46025"
    import_buoy_realtime_wave_detail(station_id)

In [ ]:
df = run_import_buoy_realtime_wave_detail()

In [ ]:
#export
t1 = PythonOperator(
    task_id='import_buoy_realtime_wave_detail',
    python_callable=run_import_buoy_realtime_wave_detail,
    dag=dag)

In [ ]:
t1.execute(context={})

Slack is broken in 1.10.7, fix is in 1.10.8.
FIXME: upgrade to 1.10.8 when its out.

In [ ]:
!airflow list_dags

[2020-02-07 09:33:48,241] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=4855
[2020-02-07 09:33:48,768] {__init__.py:51} INFO - Using executor LocalExecutor
[2020-02-07 09:33:48,769] {dagbag.py:403} INFO - Filling up the DagBag from /home/yipstar/airflow/dags


-------------------------------------------------------------------
DAGS
-------------------------------------------------------------------
huey_import_buoy_data



In [ ]:
!airflow list_tasks huey_import_buoy_data

[2020-02-07 09:33:51,318] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=4893
[2020-02-07 09:33:51,849] {__init__.py:51} INFO - Using executor LocalExecutor
[2020-02-07 09:33:51,849] {dagbag.py:403} INFO - Filling up the DagBag from /home/yipstar/airflow/dags
import_buoy_raw_spectral_wave_data
import_buoy_realtime_wave_detail


# import_buoy_raw_spectral_wave_data

In [ ]:
#export
from huey.data.importer.buoy_data import import_buoy_raw_spectral_wave_data

def run_import_buoy_raw_spectral_wave_data():
    station_id = "46025"
    import_buoy_raw_spectral_wave_data(station_id)

In [ ]:
data = run_import_buoy_raw_spectral_wave_data()

[2020-01-22 13:33:14,079] {base_hook.py:84} INFO - Using connection to: id: huey_dev. Host: localhost, Port: None, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: None
observation for date: 2020-01-22 21:00:00+00:00 already present, skipping.
import complete


In [ ]:
#export
t2 = PythonOperator(
    task_id='import_buoy_raw_spectral_wave_data',
    python_callable=run_import_buoy_raw_spectral_wave_data,
    dag=dag)

In [ ]:
t2.execute(context={})

observation for date: 2020-01-22 21:00:00+00:00 already present, skipping.
import complete


# Slack Alert

In [ ]:
#export
slack_token = BaseHook.get_connection('slack').password
t3 = SlackAPIPostOperator(
    task_id='send_slack_success_alert',
    token=slack_token,
    text='Buoy Data Import Success.',
    channel='#huey_data_import',
    username='heyhueyapp',
    dag=dag,
    blocks=[],
    attachments=[]
)

/home/yipstar/miniconda3/envs/prophet/lib/python3.7/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as huey.data.models.buoy.Buoy, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
/home/yipstar/miniconda3/envs/prophet/lib/python3.7/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as huey.data.models.buoy.BuoyRawSpectralWaveData, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
[autoreload of huey.data.models.buoy failed: Traceback (most recent call last):
  File "/home/yipstar/miniconda3/envs/prophet/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/yipstar/miniconda3/envs/prophet/lib/python3.7/site-packa

In [ ]:
t3.execute(context={})

# Export

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.sql.ipynb.
Converted 02_data.importer.buoy_data.ipynb.
Converted 03_dag.buoy_data.ipynb.
Converted 04_data.models.buoy.ipynb.


In [ ]:
from huey.dag.buoy_data import run_import_buoy_realtime_wave_detail
run_import_buoy_realtime_wave_detail()

In [ ]:
!airflow list_dags

[2020-01-22 13:33:55,859] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=31908
[2020-01-22 13:33:56,440] {__init__.py:51} INFO - Using executor SequentialExecutor
[2020-01-22 13:33:56,441] {dagbag.py:403} INFO - Filling up the DagBag from /home/yipstar/airflow/dags


-------------------------------------------------------------------
DAGS
-------------------------------------------------------------------
huey_import_buoy_data



In [ ]:
!airflow list_tasks huey_import_buoy_data

[2020-01-22 13:34:01,417] {settings.py:254} INFO - settings.configure_orm(): Using pool settings. pool_size=5, max_overflow=10, pool_recycle=1800, pid=31947
[2020-01-22 13:34:01,945] {__init__.py:51} INFO - Using executor SequentialExecutor
[2020-01-22 13:34:01,945] {dagbag.py:403} INFO - Filling up the DagBag from /home/yipstar/airflow/dags
import_buoy_raw_spectral_wave_data
import_buoy_realtime_wave_detail
